## NoteBook for the first project
- Je pensais que l'on pouvait essayer de faire ca comme ceci:
    - Chacun de nous a un block (peut bien entendu en ajouter en dessous du sien) afin de ne pas avoir trop de conflit lorsque l'on git push
        - A tester cette semaine pour voir si c'est ok

# Stefan's Blocks

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from proj1_helpers import * 
from play_with_data import * # Contains some functions that allow us to look the data and manipulate a little bit
from pre_processing import * # Contains all the functions needed to do some pre-processing on the data
%load_ext autoreload
%autoreload 2



In [ ]:
# ********** Import csv file ***********

# Import of train.csv --> This was my way of doing it
# id_train, labels_train, features_train = load_data("train.csv")

# By using the helpers 
data_path = "train.csv"
labels_train, features_train, id_train = load_csv_data(data_path)

# Import of test.csv --> This is my way of doing it, because it is not done in proj1_helpers
id_test, features_test = load_data("test.csv")

In [ ]:
# ********* Data whitening and "build model" ***********
# I think the order of the two functions below is important, because if we apply the whitening on the column of 1 it doesn't make sense I thin, do you???????

features_train = data_whitening(features_train) # to whiten the data
# Create the data for the model --> add a column of "1" as the first column --> this is for the w0 (offset parameter)
features_train = build_model_data(features_train, labels_train)[1] # the "[1]" is to take only the "X" matrix and not the "y"

In [ ]:
features_bad_ind = np.unique(np.where(features_train == -999.0)[1])
features_train = np.delete(features_train,features_bad_ind,1)
print(features_train.shape)

In [ ]:
for i in range(features_train.shape[1]):
    for j in list(range(i + 1)):    
        newFeature = np.multiply(features_train[:,i],features_train[:,j])
        features_train = np.c_[features_train,newFeature]


In [ ]:
print(a.shape)

#### Do a cross validation with logistic_regression on the data without the features with -999.0

#### Hyperparameters:
- Gamma
- K --> number of sets for the cross validation 

In [ ]:
from cross_val import *

# Prepare the data, remove the features with at least one value at -999.0
data_features_removed = remove_features(features_train)

seed = 1
degree = 5 # No more useful
k_fold = 15 # I have seen a decreasing in the error when increasing the number of fold
gammas = np.linspace(-0.5,0.5,10)
#lambdas = np.logspace(-4, 7, 40)# No longer useful because we don't use anymore the ridge regression


# split data in k fold
k_indices = build_k_indices(labels_train, k_fold, seed)
# define lists to store the loss of training data and test data
allValidationError = []

for g in gammas: # To see with which gamma we obtained a better result
    validation_error = 0
    for k in range(k_fold):
        validation_error = validation_error + cross_validation(labels_train, data_features_removed, k_indices, k, g) 
    allValidationError.append(validation_error/k_fold) # we compute the mean of the validation error
    


In [ ]:
# In order to have some information about the cross validation we just did
plt.close("all")
plt.plot(gammas, allValidationError )
print(allValidationError)
print(min(allValidationError))
plt.show()

# FUNCTIONS --> but not so sure of them! Maybe need to re-do (I can put the one I've did from the exercise, I'll do this Friday afternon)

In [ ]:
# To compute the losse, with mse --> with solution
def compute_mse(y, tx, w):
    """Calculate the loss.

    You can calculate the loss using mse or mae.
    """
    """compute the loss by mse."""
    e = y - tx.dot(w)
    mse = e.dot(e) / (2 * len(e)) # Here if we have any problem we should try with a transpose
    return mse

In [ ]:
# Gradient descent

def compute_gradient(y, tx, w):
    """Compute the gradient."""

    # compute gradient and loss
    e = y - np.dot(tx, w)
    gradient = -1/len(y) * np.dot(tx.T, e)
    
    loss = compute_mse(y,tx,w)
    return gradient, loss


def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w 
    for n_iter in range(max_iters):
        # compute gradient and loss
        gradient, loss = compute_gradient(y,tx,w)

        # update w by gradient
        w = ws[n_iter] - gamma*gradient
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses[-1], ws[-1,:]

In [ ]:
# Stochastic gradient descent
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]
            

def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    e = y - np.dot(tx, w)
    gradient = -1/len(y) * np.dot(tx.T, e)
    
    loss = compute_loss(y,tx,w)
    return gradient, loss


def stochastic_gradient_descent(y, tx, initial_w, max_iters, gamma, batch_size=1):
    """Stochastic gradient descent algorithm."""
    ws = [initial_w]
    losses = []
    lossForOneRun = 0
    
    for n_iters in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gradient, loss = compute_stoch_gradient(minibatch_y, minibatch_tx, ws[n_iters])
            lossForOneRun = lossForOneRun + loss       #Faudrait pas reinitialiser avant chaque iteration
            w = ws[n_iters] - gamma*gradient
        
        losses.append((1/batch_size)*lossForOneRun)
        ws.append(w)
    
    return losses[-1], ws[-1,:]

In [ ]:
# Least squares --> With solution
def least_squares(y, tx):
    """calculate the least squares solution."""
    
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)
    loss = compute_mse(y, tx, w)
    
    return w, loss


In [ ]:
# Ridge_regression using normal equations --> with solution
def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""

    aI = 2 * tx.shape[0] * lambda_*np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    
    w = np.linalg.solve(a,b)
    loss = compute_mse(y, tx, w)
    
    return w, loss

# Etienne's Blocks

In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from helpers import *
%load_ext autoreload
%autoreload 2


In [ ]:
trainData = pd.read_csv("train.csv")
testData = pd.read_csv("test.csv")
display(trainData.head(5))
display(testData.head(5))

In [ ]:
trainData = trainData.drop(['Id','Prediction'], 1)

In [ ]:
'''
Visualization of the data in order to have a first insight of the features and the distribution of non computable("NC")
values i.e -999.000
'''
NCFeatures = []
NCValues = []
for i in range(trainData.shape[1]):

        b = np.where(features_train[:,i] <= -999.000)[0]
        if (len(b) >= 1000):
            
            NCFeatures.append(i)
            NCValues.append(len(b))
            b = 0
            
        print('Feature ' + str(i) + ' Histogram')
        plt.hist(trainData.iloc[:,i])
        plt.show()         
                     

In [ ]:
df = pd.DataFrame(NCValues, index = NCFeatures)
display(df)

# Ivan's Blocks

In [60]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from proj1_helpers import * 
from play_with_data import * # Contains some functions that allow us to look the data and manipulate a little bit
from pre_processing import * # Contains all the functions needed to do some pre-processing on the data
from logistic_regression import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# ********** Import csv file ***********

# Import of train.csv --> This was my way of doing it
# id_train, labels_train, features_train = load_data("train.csv")

# By using the helpers 
data_path = "train.csv"
labels_train, features_train, id_train = load_csv_data(data_path)
labels_train[np.where(labels_train == -1)[0]] = 0

# Import of test.csv --> This is my way of doing it, because it is not done in proj1_helpers
#id_test, features_test = load_data("test.csv")

In [3]:
# ********* Data whitening and "build model" ***********
# I think the order of the two functions below is important, because if we apply the whitening on the column of 1 it doesn't make sense I thin, do you???????

features_train = data_whitening(features_train) # to whiten the data
# Create the data for the model --> add a column of "1" as the first column --> this is for the w0 (offset parameter)
features_train = build_model_data(features_train, labels_train)[1] # the "[1]" is to take only the "X" matrix and not the "y"

In [4]:
features_bad_ind = np.unique(np.where(features_train == -999.0)[1])
features_train = np.delete(features_train,features_bad_ind,1)
print(features_train.shape)

(250000, 20)


In [5]:
''' 
Adding combination of features as new features at the power of choice. Ex (x1x2)^coef, (x1x3)^coef. 
x1^pow is not calculated here.
'''
def twoFeatureCombinationPower(data, coef, featNum):
    
    for i in range(featNum):
        for j in list(range(i + 1)):
            if i!=j:
                newFeature = np.multiply(data[:,i],data[:,j])
                for k in range(coef):
                    newFeaturePow = np.power(newFeature,k+1)
                    data = np.c_[data,newFeaturePow]
    return data

In [6]:
''' 
Adding combination of features as new features. Ex x1^2 is calculated here
'''
def FeaturePower(data, coef,featNum):
    if coef <= 1:
        print('No need to use this function you will duplicate features')
    else:    
        powers = np.linspace(2,coef,coef-1)    
        for i in range(featNum):        
            newFeature = data[:,i]
            for j in powers:
                newFeaturePow = np.power(newFeature,j)
                data = np.c_[data,newFeaturePow]
                
    return data

In [7]:
tempData = features_train
data = twoFeatureCombinationPower(tempData,1,features_train.shape[1])
print(data.shape)
data = FeaturePower(data,2,features_train.shape[1])
print(data.shape)

(250000, 210)
(250000, 230)


In [73]:
y=labels_train[110000:130000]
tx=data[110000:130000,:]
initial_w=np.zeros((tx.shape[1], 1))
max_iter=100
gamma=1
lambda_=100


In [29]:
seed = 1
k_fold = 15
gammas = np.linspace(0.25,1,4) # Hyperparameter
lambdas = np.logspace(-1,8,10) # Hyperparameter

k_indices = build_k_indices(y, k_fold, seed)
allValidationError = np.zeros([len(gammas), len(lambdas)])

for ind_g, g in enumerate(gammas):
    for ind_l, l in enumerate(lambdas):
        validation_error = 0
        print("gamma: {}\tlambda: {}".format(g,l))
        for k in range(k_fold):
            print(k)
            validation_error = validation_error + cross_validation(y, tx, k_indices, k, g, l)
        allValidationError[ind_g, ind_l] = validation_error/k_fold

In [30]:
w,loss=learning_by_penalized_gradient(y, tx, initial_w, max_iter, gamma, lambda_)

Current iteration=0, loss=[[ 13862.9436112]]
Current iteration=1, loss=[[ 9565.5228435]]
Current iteration=2, loss=[[ 8957.96359879]]
Current iteration=3, loss=[[ 8739.83258948]]
Current iteration=4, loss=[[ 8645.15851427]]
Current iteration=5, loss=[[ 8604.02139854]]
Current iteration=6, loss=[[ 8592.5321059]]
Current iteration=7, loss=[[ 8592.05446394]]
Current iteration=8, loss=[[ 8592.05285207]]


In [76]:
batch_size=7000
w,loss=penalized_stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iter, gamma, lambda_)
print(w.shape)

Gradient Descent(0/99): loss=[[ 4852.03026392]]
Gradient Descent(1/99): loss=[[ 3898.78117069]]
Gradient Descent(2/99): loss=[[ 3603.57749163]]
Gradient Descent(3/99): loss=[[ 3352.98716923]]
Gradient Descent(4/99): loss=[[ 3363.00089446]]
Gradient Descent(5/99): loss=[[ 3310.79859485]]
Gradient Descent(6/99): loss=[[ 3334.30797855]]
Gradient Descent(7/99): loss=[[ 3256.72333829]]
Gradient Descent(8/99): loss=[[ 3261.99644048]]
Gradient Descent(9/99): loss=[[ 3365.34729899]]
Gradient Descent(10/99): loss=[[ 3366.32117287]]
Gradient Descent(11/99): loss=[[ 3330.98818235]]
Gradient Descent(12/99): loss=[[ 3220.28877737]]
Gradient Descent(13/99): loss=[[ 3212.35117668]]
Gradient Descent(14/99): loss=[[ 3283.37793011]]
Gradient Descent(15/99): loss=[[ 3265.47454818]]
Gradient Descent(16/99): loss=[[ 3266.01044894]]
Gradient Descent(17/99): loss=[[ 3209.82841505]]
Gradient Descent(18/99): loss=[[ 3247.23097114]]
Gradient Descent(19/99): loss=[[ 3242.4052503]]
Gradient Descent(20/99): loss=[

KeyboardInterrupt: 

## Rendering submission

In [ ]:
id_test, features_test = load_data("test.csv")

features_train = data_whitening(features_test) # to whiten the data
features_train = nonComputableValuesIntegration(features_test)
# Create the data for the model --> add a column of "1" as the first column --> this is for the w0 (offset parameter)
features_test = build_model_data(features_test) # the "[1]" is to take only the "X" matrix and not the "y"

tempData = features_test
data = twoFeatureCombinationPower(tempData,1,features_test.shape[1])
data = FeaturePower(data,2,features_test.shape[1])


y_pred = predict_labels(w1, data[:,:])
create_csv_submission(id_test, y_pred, 'first_submission')